In [18]:
import boto
import boto.ec2
import ConfigParser
import base64
import boto.sqs
import boto.s3.connection
import boto.dynamodb
import shutil
import os

In [19]:
#read config file
def read_config_file(file_name):
    config = ConfigParser.RawConfigParser()
    config.read(file_name)
    return config

In [20]:
config = read_config_file('../config/config.cfg')

In [21]:
#sqs connection 
connsqs = boto.sqs.connect_to_region( config.get('AWS_CRED','REGION'),
                                   aws_access_key_id= config.get('AWS_CRED','AWS_ACCESS_KEY_ID'),
                                   aws_secret_access_key=config.get('AWS_CRED','AWS_SECRECT_ACCESS_KEY'))

In [22]:
#create queue
q = connsqs.create_queue(config.get('SQS','QUEUE_NAME'), 
                          config.get('SQS','TIME_OUT'))

In [23]:
#dynamo DB connection
conndynamo = boto.dynamodb.connect_to_region( config.get('AWS_CRED','REGION'),
                                   aws_access_key_id= config.get('AWS_CRED','AWS_ACCESS_KEY_ID'),
                                   aws_secret_access_key=config.get('AWS_CRED','AWS_SECRECT_ACCESS_KEY'))

In [24]:
#create schema
message_table_schema = conndynamo.create_schema(
        hash_key_name=config.get('DYNAMODB','HASH_KEY'),
        hash_key_proto_value=str,
    )

In [25]:
#create table
table = conndynamo.create_table(
        name= config.get('DYNAMODB','TABLE_NAME'),
        schema=message_table_schema,
        read_units= long(config.get('DYNAMODB','READ_UNITS')),
        write_units=long(config.get('DYNAMODB','WRITE_UNITS'))
    )

In [9]:
# %run D:/Cloud1/animato_master/webserver/webserver_2.py

In [26]:
#get connection ec2
conn = boto.ec2.connect_to_region(config.get('AWS_CRED','REGION'),
                                  aws_access_key_id= config.get('AWS_CRED','AWS_ACCESS_KEY_ID'),
                                  aws_secret_access_key=config.get('AWS_CRED','AWS_SECRECT_ACCESS_KEY'))

In [27]:
#create security group for worker
worker_security_grp = conn.create_security_group(config.get('WORKER','SEQURITY_GROUP_NAME'),
                                 config.get('WORKER','SEQURITY_GROUP_NAME_DESC'))

worker_security_grp.authorize('tcp', 22, 22, '0.0.0.0/0')

True

In [12]:
'''
start_worker_code = """#!/bin/sh
python /home/ec2-user/animato_master/worker/worker.py
"""
'''

'\nstart_worker_code = """#!/bin/sh\npython /home/ec2-user/animato_master/worker/worker.py\n"""\n'

In [13]:
'''
#launch worker instance
conn.run_instances(config.get('WORKER','IMAGE_ID'),
        key_name=  config.get('AWS_CRED','KEY_NAME'),
        instance_type= config.get('WORKER','INSTANCE_TYPE'),
        security_groups= [worker_security_grp],
        user_data = start_worker_code
        )
'''

"\n#launch worker instance\nconn.run_instances(config.get('WORKER','IMAGE_ID'),\n        key_name=  config.get('AWS_CRED','KEY_NAME'),\n        instance_type= config.get('WORKER','INSTANCE_TYPE'),\n        security_groups= [worker_security_grp],\n        user_data = start_worker_code\n        )\n"

In [28]:
#create security group for watcher
watcher_security_grp = conn.create_security_group(config.get('WATCHER','SEQURITY_GROUP_NAME'),
                                 config.get('WATCHER','SEQURITY_GROUP_NAME_DESC'))

watcher_security_grp.authorize('tcp', 22, 22, '0.0.0.0/0')

True

In [29]:
start_watcher_code = """#!/bin/sh
python /home/ec2-user/cloud_master/watcher/watcher.py
"""

In [30]:
#launch watcher instance
conn.run_instances(config.get('WATCHER','IMAGE_ID'),
        key_name=  config.get('AWS_CRED','KEY_NAME'),
        instance_type= config.get('WATCHER','INSTANCE_TYPE'),
        security_groups= [watcher_security_grp],
        user_data = start_watcher_code
        )

Reservation:r-083d4a5512cf38eb4